## NOTEBOOK IS NOW DONE

This notebook is now done... 
The database is built, and most queries can be done using SQL language, so I make a new notebook for examples.

In [1]:
cd ..

C:\Users\csav6865\Documents\GitHub\picarro-calculator


In [2]:
import Picarro as pica
import PicarroDatabase as pdb
import pickle

In [3]:
files = pdb.checkforrawdata(r"J:\c715\Picarro\Results\Results 2019\Raw data")

There is no new raw data in directory: J:\c715\Picarro\Results\Results 2019\Raw data


In [4]:
conn = pdb.CreateConnection(r"J:\c715\Picarro\Results\Database\data.db")

for file in files:
    pdb.AddRaw(file,conn)
    try:
        pdb.AddSummaryRun(file,conn)
    except:
        print("An error occurred")

TypeError: 'NoneType' object is not iterable

In [5]:
import numpy as np
import glob
import pandas as pd

Here, we've selected the run ids and their respective nicknames for the year 2019

In [64]:
df = pd.read_sql_query("select * from runs r where r.'Identifier 2' = 'TP';",conn)
df

,key,Identifier 1,Identifier 2,RUN_ID,position,d18O vsmow,d18O stdev. vsmow,d18O counts,d2H vsmow,d2H stdev. vsmow,d2H counts,inside GMWL
0,9524,LAQU01,TP,20190416,25,-12.336111,0.039799,4,-85.621242,0.196441,4,inside
1,9525,LAQU02,TP,20190416,26,-12.222082,0.045511,4,-85.167786,0.285575,4,inside
2,9597,ZNLO1,TP,20190419,17,-12.570108,0.043646,4,-88.711723,0.307661,4,inside
3,11359,QT03,TP,20191120,24,-13.256535,0.016087,3,-93.954580,0.311648,4,inside
4,11360,LAQ03,TP,20191120,25,-13.326152,0.018158,3,-94.089591,0.246735,3,inside
5,11361,ZNL03,TP,20191120,26,-12.918666,0.060837,4,-91.314351,0.318675,4,inside
6,11379,QT-26-08,TP,20191121,17,-12.965225,0.034977,4,-91.764172,0.129866,4,inside
7,11381,LAQ-26-08,TP,20191121,19,-11.925454,0.037057,3,-83.268446,0.090499,3,inside


In [67]:
df.to_csv(r'I:\Isotopedata_TP.csv')

In [41]:
df = pd.read_sql_query("select * from rawrun rw where rw.'Identifier 2'='YD';",conn)
df

,Line,Analysis,Time Code,Port,Inj Nr,d(18_16)Mean,d(D_H)Mean,H2O_Mean,Ignore,Good,...,h16od_adjust,h16od_shift,n2_flag,DAS Temp,Tray,Sample,Job,Method,Error Code,RUN_ID


we can also select the runs based on what the name is like, for example: guffert runs.

In [35]:
### In case the database is rebuilt, and the run look up table needs to be done again, uncomment the next bit of code.


#strips = ["~$","J:\\c715\\Picarro\\Results\\Results 2019\\Picarro-Results-",".xlsx","-","_"]

#nicknames = []

#for i in glob.glob(r"J:\c715\Picarro\Results\Results 2019\*.xlsx"):
#    full = i
#    for j in strips:

#        full = full.replace(j," ")
#    #print(full)
#    if full.startswith(" 2019") == True:
#        nicknames.append(full)
    
#RUN_IDS = glob.glob(r"J:\c715\Picarro\Results\Results 2019\Raw data\*.csv")

#for i,j in zip(nicknames,RUN_IDS):
#    pdb.AddRun(j,i,conn)

86

20190108.0
table runlookup already exists
20190109.0
table runlookup already exists
20190110.0
table runlookup already exists
20190111.0
table runlookup already exists
20190112.0
table runlookup already exists
20190113.0
table runlookup already exists
20190405.0
table runlookup already exists
20190406.0
table runlookup already exists
20190408.0
table runlookup already exists
20190409.0
table runlookup already exists
20190410.0
table runlookup already exists
20190411.0
table runlookup already exists
20190412.0
table runlookup already exists
20190413.0
table runlookup already exists
20190415.0
table runlookup already exists
20190416.0
table runlookup already exists
20190417.0
table runlookup already exists
20190418.0
table runlookup already exists
20190419.0
table runlookup already exists
20190513.0
table runlookup already exists
20190514.0
table runlookup already exists
20190515.0
table runlookup already exists
20190516.0
table runlookup already exists
20190701.0
table runlookup already

In [ ]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

Now let's execute a statement where the NickName is replaced.

In [ ]:
import sqlite3
from sqlite3 import Error

def ReplaceName(conn,RUN_ID,newname):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param RUN_ID: the run id, an eight digit integer with format yyyymmdd
    :return: updated table name in 

    https://www.sqlitetutorial.net/sqlite-python/create-tables/
    """
    
    statement= """UPDATE runlookup
    SET RUN_ID = {0}, NickName = '{1}'
    WHERE RUN_ID = {0};""".format(RUN_ID,newname)
    
    try:
        c = conn.cursor()

        c.execute(statement)
    except Error as e:
            print(e)
        conn.commit()

In [ ]:
ids = [20190513,20190514,20190515,20190516,20190707]
newnames=['2019 05 14 TR Hundsalm Run01',
         '2019 05 15 TR Hundsalm Run02',
         '2019 05 16 TR Hundsalm Run03',
         '2019 05 17 TR Hundsalm Run04',
         '2019 07 08 TR Hundsalm Run05']
for i,j in zip(ids,newnames):
    ReplaceName(conn,i,j)
    



In [ ]:
df = pd.read_sql_query("select * from runlookup rlk where rlk.NickName like '%Hundsalm%';",conn)
df

In [ ]:
pdb.AddSummaryRun(r"J:\c715\Picarro\Results\Results 2019\Raw data\HKDS2009_IsoWater_20190516_142104.csv",conn)

In [ ]:
statement ="""SELECT * from runlookup rlk,runs r 
            WHERE (
            rlk.NickName like '%Hundsalm%'
            AND (r.'Identifier 2' = 'TR' 
            AND r.RUN_ID= rlk.RUN_ID
            ));"""

df = pd.read_sql_query(statement,conn)
df

In [ ]:
df = pd.read_sql_query("select r.'Identifier 1',r.'Identifier 2' from rawrun r where RUN_ID = 20190513",conn)

In [ ]:
df1 = pd.read_csv(r"J:\c715\Picarro\Results\Results 2019\Raw data\HKDS2009_IsoWater_20190513_142143.csv")

In [ ]:
df1.head(50)

In [ ]:
pd.read_sql_query("select * from runlookup rlk,runs r where (rlk.NickName like '%Hundsalm%' and (r.RUN_ID = rlk.RUN_ID and r.'Identifier 2'  like '%Hundsalm%'));",conn)

In [ ]:
pd.read_sql_query("select * from runs where")

In [ ]:
pd.read_sql_query("select * from runs",conn)